In [ ]:
import numpy as np
import librosa
import IPython.display as ipd
import pandas as pd

In [ ]:
data = pd.read_csv("C:/Users/aelbeji/Desktop/Abir's internship/datasets_csv/iOS_audioset.csv") #Android_audioset for 3gp audios

In [ ]:
data1 = data.loc[data['audio_type'] == 'type1'] #for type1 audios
data2 = data.loc[data['audio_type'] == 'type2'] #for type2 audios

In [ ]:
#extract and store type1 and type2 audios into numpy arrays separately
#Here, only type1 audios were processed. 

sound_data1 = []
for file_name in data1['ID_short']:
    current_sound,_ =  librosa.load("Insert your PATH/Predi_Covid_audios/{}.wav".format(file_name), sr=8000) 
    sound_data1.append(current_sound)  
sound_data1 = np.array(sound_data1, dtype=object)

#np.save("sound_data1_m4a_3.npy", sound_data) #you can save your audios #sound_data2_m4a for type2 audios

In [ ]:
#wave plot vizualization
librosa.display.waveplot(sound_data1[1096], sr=8000) #example for participant with fatigue

In [ ]:
#Append silence to type1 and type2 audios separately to reach 50 seconds

target_wav_time  = 50
final_comb1 = []
for x in range(0, len(sound_data1)):
    dur = librosa.get_duration(y=sound_data1[x], sr= 8000)
    if dur > 50:
        tar=target_wav_time*8000
        combined_segment=sound_data1[x][:tar]
    else:
        silence_duration = (target_wav_time - dur)*8000  #define the duration of silence to be added to reach 50s
        combined_segment = np.hstack((sound_data1[x],np.zeros(int(silence_duration)))) 
    final_comb1.append(combined_segment)
final_comb1 = np.array(final_comb1, dtype=object)  

#np.save("audio1_m4a_3.npy", final_comb1) #audio2_m4a for type2 audios

In [ ]:
#Loading type 1 audios wiht silence
audios_type1 = np.load("Insert your PATH/sound_data1_m4a.npy",allow_pickle=True)

#Loading type 2 audios with silence
audios_type2 = np.load("Insert your PATH/sound_data2_m4a.npy",allow_pickle=True)

In [ ]:
#Concatenate type 1 and type 2 audios for each participant 

Audios_final = []
for x, y in zip(audios_type1, audios_type2):
    combined_segment = np.hstack((x,y))
    Audios_final.append(combined_segment)
Audios_final = np.array(Audios_final, dtype=object)
#saving final audios
np.save("final_sound_data_m4a.npy", Audios_final) #save concatenated type1&2 audios

In [ ]:
#check the silence&concatenation
ipd.Audio(Audios_final[0], rate= 8000)

In [ ]:
#Generating mel-spectrograms

mel_spectrograms = []
for x in Audios_final:
    mel_spectrogram_ = librosa.feature.melspectrogram(x, sr=8000, n_fft=8192, hop_length=4096, n_mels = 196) 
    log_mel_spectrogram_ = librosa.power_to_db(mel_spectrogram_)
    mel_spectrograms.append(log_mel_spectrogram_)

In [ ]:
#visualization of mel-spectrograms

import librosa.display

plt.figure(figsize=(25,10))
ml = librosa.display.specshow(mel_spectrograms[1096],  #exmaple for participant with fatigue
                         x_axis="time",
                         y_axis="mel", 
                         sr=8000)
#ml.figure.savefig("test.jpg")
#plt.colorbar(format="%+2.f")

plt.show()

In [ ]:
#check the squared shape of mel-spectrograms (196*196)

mel_spectrograms = np.array(mel_spectrograms, dtype=float)
mel_spectrograms.shape

In [ ]:
#np.save("Insert you PATH/melspectrograms_m4a.npy", mel_spectrograms) #save final mel-spectrograms